# 뉴스 빅데이터 분석 데이터 추출 & 사용자 사전 추가

In [ ]:
def Extract_Keyword_from_news_bigdata(path):
    from glob import glob
    fileList = glob(path+'*.csv')
    listIs =list()
    for i in fileList:
        df = pd.read_csv(i,engine='python')
        f = list(filter(lambda x: x[1] in ['중요키워드','키워드','keword','토픽키워드','토픽 키워드'], enumerate(df.columns.values)))
        if not len(f) == 0:
            listIs += list(df[df.columns[f[0][0]]].values)
    outlist = []
    for ix in list(filter(lambda x: type(x)!=float, listIs)):
        outlist += ix.split(',')
    outlist1 = list(map(lambda x: ''.join(x.split('_')), outlist))
    outlist2 = list(map(lambda x: ' '.join(x.split('_')), outlist))
    return outlist1+outlist2

In [ ]:
dbpath = 'C:/Users/pc/Anaconda3/lib/site-packages/ckonlpy/data/twitter/noun/'

In [ ]:
path1 = './data/nnews/high_frequency_noun/'
path2 = './data/news/mainissue/'
path3 = './data/news/newstopic/'
path4 = './data/news/people/'
path5 = './data/news/4th_industry/'
path6 = './data/news/have_negative_positive/constitution/'
path7 = './data/news/have_negative_positive/household_debt/'
path8 = './data/news/have_negative_positive/olymphic/'
path9 = './data/news/have_negative_positive/'
path10 = './data/news/'

pathlist = [path1, path2, path3,
            path4, path5, path6,
            path7, path8, path9,
           path10]
if not os.path.isfile(dbpath+'from_news.txt'):
    with open(dbpath+'from_news.txt','w') as f:
        for path in pathlist:
            out = Extract_Keyword_from_news_bigdata(path)
            f.write('\n'.join(out))

In [28]:
import pandas as pd 
from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Mecab, Twitter
from konlpy.utils import pprint
import os
def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwords = Stopwords('./data/koreanStopwords.txt') +Stopwords('./data/newspress.txt')

In [29]:
mecab = Mecab()
ct = ctwitter()
ot = Twitter()
xxxx = ct._dictionary._pos2words
list(map(lambda x: len(xxxx[x]), xxxx))

[512, 5068471, 5532]

# 자연어 처리

# 분석

In [87]:
# -*- coding: utf-8 -*-
import sys
sys.path.append('/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/')
from collections import defaultdict
from datetime import datetime, timedelta
import time, re, pickle, itertools
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
import urllib3, json
import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
import os

def OpenAPI():
    import pickle
    openapi = pickle.load(open('./etri_open_api_access_key', 'rb'))
    return openapi
def USE_ETRI_ANALYSIS(analysisCode, text):    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    accessKey = OpenAPI().strip()
    requestJson = {
        'access_key' : accessKey,
        'argument' : {
            'text' : text,
            'analysis_code' : analysisCode
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    return json.loads(str(response.data,'utf-8')), str(response.status)

def USE_ETRI_INFO(fWord, sWord):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/WordRel"
    accessKey = OpenAPI().strip()
    firstWord = fWord
    #firstSenseId = fSenseId
    secondWord = sWord
    #secondSenseId = sSenseId
    requestJson = {
        "access_key": accessKey,
        "argument": {
        'first_word': firstWord,
        #'first_sense_id': firstSenseId,
        'second_word': secondWord,
        #'second_sense_id': secondSenseId
        }}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8')), str(response.status)

def USE_ETRI_RELATION(word):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/Word"
    accessKey = OpenAPI().strip()
    word = word
    requestJson = {
    "access_key": accessKey,
    "argument": {"word": word}}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
                            body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8')), str(response.status)

def Extract_Text_Info(idx):
    morp = list(map(lambda x: (x['lemma'], x['type']),idx['morp']))
    wsd = list(map(lambda x: (x['text'], x['type']),idx['WSD']))
    word = list(map(lambda x: (x['text'], x['type']),idx['word']))
    ne = list(map(lambda x: (x['text'], x['type']),idx['NE']))
    return morp, wsd, word, ne

def Run_ETRI_Analysis(stopwordsList, text):
    etri = USE_ETRI_ANALYSIS('srl', text)['return_object']    
    etri = etri['sentence']
    morp = [] ; wsd = [] ; word = [] ; ne = [] ; srl = [] ; dependency = []
    for i in etri:
        x = Extract_Text_Info(i)
        morp += x[0]
        wsd += x[1]
        word +=x[2]
        ne += x[3]
        for ii in i['SRL']:
            srl += list(map(lambda x: (x['text'], x['type']),ii['argument']))
        dependency += list(map(lambda x: (x['text'], x['label']), i['dependency']))
    return morp, wsd, word, ne, srl, dependency, etri
    
if __name__ == '__main__':
    pass

In [ ]:
from lexrankr import LexRank
lexrank = LexRank()
lexrank.summarize()
lr = lexrank.probe(k=3)

In [655]:
class SentenceTokenizer(object):
    def __init__(self):
        self.twitter = ct
        #self.twitter = ot
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
        ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]    
    def text2sentences(self, text):
        sentences = re.sub('\. ','.\n', text).split('\n')
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                #nouns.append(' '.join([noun for noun in self.twitter.phrases(str(sentence))
                #                                    if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
        return nouns
    
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = sknorm(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=4):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords

In [674]:
def Keyword(etri_outcome, data):
    testdata = pickle.load(open(etri_outcome+data['_id']._ObjectId__id.hex()+'.picked.txt','rb'))
    etri = testdata['return_object']['sentence']
    morp = [] ; wsd = [] ; word = [] ; ne = [] ; srl = [] ; dependency = []
    for i in etri:
        x = Extract_Text_Info(i)
        morp += x[0]
        wsd += x[1]
        word +=x[2]
        ne += x[3]
        for ii in i['SRL']:
            srl += list(map(lambda x: (x['text'], x['type']),ii['argument']))
        dependency += list(map(lambda x: (x['text'], x['label']), i['dependency']))
    morp1 = list(map(lambda x: x[0], morp))
    wsd1 = list(map(lambda x: x[0], wsd))
    word1 = list(map(lambda x: x[0], word))
    ne1 = list(map(lambda x: x[0], ne))
    srl1 = list(map(lambda x: x[0], srl))
    dependency1 = list(map(lambda x: x[0], dependency))
    outlist = []
    for idx in range(len(etri)):
        y = pd.DataFrame(etri[idx]['dependency'])
        y2 = y[y['mod'].apply(lambda x: len(x)) ==y['mod'].apply(lambda x: len(x)).max()]
        y3 = y2[y2['weight'] == y2['weight'].max()]
        y4 = y[y['id'].isin(y3['mod'][y3['mod'].index[0]])]
        y5 = y[y['weight'] ==y4['weight'].max()]
        y6 = y2.text.values.tolist() + y5.text.values.tolist()
        y7 = y[y.text.isin(y6)]    
        out = ' '.join(y7.text.tolist())
        outlist.append(out)
    sText = data['title']+'. '+data['mainText']
    sText = '.\n'.join(sText.split('. '))
    vect = TfidfVectorizer(ngram_range=(1,2),
                      norm='l2',
                      stop_words=stopwords)
    vect.fit(morp1)
    vect.fit(wsd1)
    vect.fit(word1)
    vect.fit(ne1)
    vect.fit(srl1)
    vect.fit(outlist)
    vect.fit(dependency1)
    #vect.fit(ct.nouns(sText))
    count1 = vect.fit_transform([sText]).toarray().sum(axis = 0)
    ch = ct.nouns(data['title'])+ct.phrases(data['title'])+outlist
    ch2 = list(map(lambda x: re.sub('[\W]',' ', x), ch))
    ch2 = list(map(lambda x: x.strip(), ch2))
    ch3 = list(filter(lambda x: x[1] in ch2, enumerate(vect.get_feature_names())))
    y =  [1] * len(vect.get_feature_names())
    for ix in ch3:
        y[ix[0]] = 1.5
    count1 = count1 * y
    idx1 = np.argsort(-count1)
    count1 = count1[idx1]
    feature_name1 = np.array(vect.get_feature_names())[idx1]
    out1 = dict(zip(feature_name1, count1))
    return sorted(out1.items(), key=lambda x: x[1], reverse=True)[:7]
    

In [675]:
#site = 'Naver'
site = 'daum'
if site == 'daum':
    collection = 'newsDaum'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/daum2/'
elif site.lower() == 'naver':
    collection = 'newsNaver'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/naver2/'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, collection)

keywordList = list()
pressList = list()
for data in useCollection.find({'site':site}):
    pressList.append(data['press'])
    tis = data['title']+'. '+data['mainText']
    tis2 = '.\n'.join(tis.split('. '))
    if len(tis) != 0:
        if data['keywords'] !='NaN':
            keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
        else:
            pass
    idIs = data['_id']._ObjectId__id
    textrank = TextRank(tis2)
    keywords = Keyword(etri_outcome,data)
    #workrank = RunWordKRwordRank(tis2, 2)
    #lexrank = LexRank()
    #lexrank.summarize(tis2)
    #lr = lexrank.probe(k=1)
    print (textrank.keywords(),keywords, data['keywords'])
    #print (lr)
    print ()
    '''
    if not os.path.isfile(etri_outcome+idIs.hex()+'.picked.txt'):
        try:
            etri = USE_ETRI_ANALYSIS('srl', tis2)
        except:
            print (etri[1])
            break
        else:
            pickle.dump(etri[0], open(etri_outcome+idIs.hex()+'.picked.txt','wb'))
    break
    '''

['한명회', '이름', '이름을', '방송'] [('한명회', 0.4845437118523489), ('이름을', 0.21535276082326621), ('막을 내린다', 0.16151457061744967), ('5일', 0.16151457061744967), ('이름', 0.1076763804116331), ('했다', 0.1076763804116331), ('막을', 0.1076763804116331)] ['노홍철', '한명회', '한혜진', '종영', '편성시간']

['캠프', '선수', '신인', '류중일'] [('류중일', 0.30292214689575725), ('있다', 0.24233771751660582), ('신인', 0.24233771751660582), ('캠프', 0.24233771751660582), ('했다', 0.20194809793050483), ('2차', 0.12116885875830291), ('없다', 0.12116885875830291)] ['류중일']

['좌윤', '상무', '비서', '으로'] [('좌윤이는', 0.26711222531767115), ('봉상무', 0.23372319715296225), ('보스', 0.20033416898825335), ('봉상무의', 0.16694514082354445), ('저글러스', 0.15025062674119), ('좌윤이를', 0.10016708449412667), ('좌윤이가', 0.10016708449412667)] ['불륜', '비', '첫방', '최다니엘', '최대철']

['기억', '장항준', '영화', '스릴러'] [('장항준', 0.35097629199405966), ('기억의', 0.26741050818595025), ('걱정을', 0.13370525409297512), ('감독이', 0.13370525409297512), ('감독은', 0.13370525409297512), ('장항준 감독이', 0.13370525409297512), ('제가', 

['변호', '이영', '이영학', '사람'] [('그는', 0.23398215222964938), ('이영학이', 0.20473438320094323), ('이영학의', 0.17548661417223704), ('있다', 0.17548661417223704), ('이영학', 0.13161496062917777), ('말했다', 0.087743307086118522), ('약을', 0.087743307086118522)] ["이슈 · '어금니 아빠' 살인사건", '이영학', '여중생', '서울구치소']

['구조', '으로', '대통령', '세월호'] [('박근혜', 0.21972288386821306), ('대통령은', 0.17903346092965508), ('대통령의', 0.16275769175423188), ('사고', 0.14648192257880871), ('대통령', 0.14648192257880871), ('세월호', 0.14648192257880871), ('지난', 0.13020615340338551)] ['이슈 · 영흥도 낚싯배 전복사고', '문재인', '박근혜', '세월호']

['복귀', '폭격', '가능성', '관건'] [('도미니카', 0.36380343755449945), ('복귀 가능성', 0.36380343755449945), ('가능성', 0.36380343755449945), ('복귀', 0.36380343755449945), ('몸값', 0.36380343755449945), ('리즈', 0.36380343755449945), ('높은 몸값', 0.36380343755449945)] NaN

['음식', '하루', '다이어트', '섭취'] [('하루', 0.32114446885847342), ('12시간', 0.1835111250619848), ('먹는', 0.15292593755165401), ('먹고', 0.12234075004132321), ('섭취', 0.12234075004132321), ('하는', 0.12234

['야구선수', '앵커', '으로', '이승엽'] [('이승엽', 0.57998658549941284), ('앵커', 0.53855897224945481), ('야구선수', 0.35903931483296986), ('이승엽 야구선수', 0.3038024971663591), ('제가', 0.13809204416652687), ('손석희', 0.1242828397498742), ('저는', 0.11047363533322149)] ['이승엽']

['효연', '캠핑', '공개', '10년'] [('캠핑', 0.36997584958692442), ('데뷔', 0.27748188719019329), ('10년', 0.18498792479346221), ('살미도', 0.18498792479346221), ('효연은', 0.18498792479346221), ('데뷔 10년', 0.18498792479346221), ('만에', 0.18498792479346221)] ['캠핑', '살짝미쳐도좋아', '눈길', '소녀시대', '반려견']

['지사', '대통령', '발언을', '지지'] [('대통령', 0.29528042171615398), ('지사의', 0.26247148596991465), ('대통령이', 0.16404467873119666), ('발언은', 0.13123574298495733), ('있는', 0.098426807238717995), ('발언을', 0.098426807238717995), ('지지자들은', 0.098426807238717995)] ['안희정', '문재인', '경선']

['앞둔', '집권', '2기', 'kia'] [('2기', 0.36380343755449945), ('앞둔', 0.36380343755449945), ('김기태 감독', 0.36380343755449945), ('집권', 0.36380343755449945), ('길이', 0.36380343755449945), ('김기태', 0.36380343755449945), ('감

['오전', '구조', '대통령', '현장'] [('지시했다', 0.19936305570722498), ('오전', 0.19936305570722498), ('낚싯배', 0.1794267501365025), ('대통령은', 0.15949044456577999), ('행안부', 0.11961783342433499), ('필요한', 0.11961783342433499), ('않도록', 0.11961783342433499)] ['이슈 · 영흥도 낚싯배 전복사고', '낚싯배', '전복사고', '세월호']

['성폭력', '이씨', '등을', '범행'] [('성폭력', 0.17349447958987207), ('며느리를', 0.17349447958987207), ('징역', 0.17349447958987207), ('아들이', 0.11566298639324804), ('밝혔다', 0.11566298639324804), ('등을', 0.11566298639324804), ('이씨는', 0.11566298639324804)] ['며느리', '성폭력범죄', '낙태']

['린드블럼', '롯데', '선수', '투수'] [('롯데', 0.18028869304837589), ('린드블럼은', 0.16025661604300079), ('린드블럼의', 0.16025661604300079), ('평균자책점', 0.12019246203225059), ('린드블럼이', 0.12019246203225059), ('것으로', 0.12019246203225059), ('받았다', 0.080128308021500394)] ['조쉬 린드블럼']

['kt', '윤석민', '황재균', '3루'] [('윤석민은', 0.21863473472331302), ('있다', 0.17490778777865043), ('kt', 0.17490778777865043), ('선발', 0.17490778777865043), ('황재균', 0.13118084083398782), ('감독은', 0.1311808408339

['온유', '지난', '팬들', '활동'] [('온유는', 0.20982172726556325), ('샤이니', 0.14987266233254518), ('온유가', 0.14987266233254518), ('팬들은', 0.14987266233254518), ('지난', 0.14987266233254518), ('대한', 0.11989812986603614), ('온유의', 0.11989812986603614)] ['자숙', '샤이니', '사과문', 'sm엔터테인먼트', '이글']

['다저스', '트레이드', '류현진', '윈터미팅'] [('트레이드', 0.40840266683181736), ('류현진', 0.17502971435649314), ('다저스', 0.15558196831688278), ('거닉', 0.15558196831688278), ('11일', 0.1166864762376621), ('류현진은', 0.1166864762376621), ('윈터미팅', 0.1166864762376621)] ['류현진']

['기사', '결별', '한혜진', '박나래'] [('결별', 0.25682856549090732), ('라고', 0.22829205821413984), ('한혜진은', 0.17121904366060489), ('나혼자산다', 0.17121904366060486), ('말해', 0.11414602910706992), ('반응을', 0.11414602910706992), ('쿨한 반응을', 0.11414602910706992)] ['한혜진', '결별', '차우찬', '전현무', '박나래']

['램지', '고든', '냉장고', '요리'] [('고든', 0.69411722007412124), ('고든 램지가', 0.20823516602223635), ('요리', 0.13882344401482427), ('램지가', 0.13882344401482424), ('램지는', 0.13882344401482424), ('고든 램지는', 0.13882344

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [550]:
def RunWordKRwordRank(text, min_count):
    #min_count = 2   # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10 # 단어의 최대 길이
    wordrank_extractor = KRWordRank(min_count, max_length)
    beta = 0.85   # PageRank의 decaying factor beta
    max_iter = 100
    verbose = False
    textIs = re.sub('\. ','.\n', text).split('\n')
    textIs = list(map(lambda x: normalize(x, english = True, number = True), textIs))
    keywords, rank, graph = wordrank_extractor.extract(textIs, beta, max_iter, verbose)
    return keywords, rank, graph

In [630]:
testdata = pickle.load(open(etri_outcome+data['_id']._ObjectId__id.hex()+'.picked.txt','rb'))
etri = testdata['return_object']['sentence']
morp = [] ; wsd = [] ; word = [] ; ne = [] ; srl = [] ; dependency = []

for i in etri:
    x = Extract_Text_Info(i)
    morp += x[0]
    wsd += x[1]
    word +=x[2]
    ne += x[3]
    for ii in i['SRL']:
        srl += list(map(lambda x: (x['text'], x['type']),ii['argument']))
    dependency += list(map(lambda x: (x['text'], x['label']), i['dependency']))
morp1 = list(map(lambda x: x[0], morp))
wsd1 = list(map(lambda x: x[0], wsd))
word1 = list(map(lambda x: x[0], word))
ne1 = list(map(lambda x: x[0], ne))
srl1 = list(map(lambda x: x[0], srl))
dependency1 = list(map(lambda x: x[0], dependency))
outlist = []
for idx in range(len(etri)):
    y = pd.DataFrame(etri[idx]['dependency'])
    y2 = y[y['mod'].apply(lambda x: len(x)) ==y['mod'].apply(lambda x: len(x)).max()]
    y3 = y2[y2['weight'] == y2['weight'].max()]
    y4 = y[y['id'].isin(y3['mod'][y3['mod'].index[0]])]
    y5 = y[y['weight'] ==y4['weight'].max()]
    y6 = y2.text.values.tolist() + y5.text.values.tolist()
    y7 = y[y.text.isin(y6)]    
    out = ' '.join(y7.text.tolist())
    outlist.append(out)

In [552]:
sTest = data['title']+'. '+data['mainText']
sTest = '.\n'.join(sTest.split('. '))
sTest

'김국진·노홍철·한혜진 \'한명회\', 오늘(5일) 9회만에 종영.\n영상 바로보기 [마이데일리 = 이승길 기자] JTBC \'내 이름을 불러줘-한명(名)회\'(이하 \'한명회\')가 막을 내린다.\n최근 진행된 \'한명회\' 9회 녹화는 별난 이름 특집으로 꾸며졌다.\n이날은 특집 명에 맞게 녹화 전까지 철저히 출연자들이 서로 이름을 공유하지 못하도록 조치를 취했다.\n출연자들은 서로의 이름을 추측하며 자신의 이름보다 특이 한 지 궁금해 했다.\n이날 8명의 출연자는 모두 "출석 부르는 시간, 자기소개 시간을 가장 싫어한다"며 다른 이름이지만 같은 심정으로 공감 대화를 나눴다.\n그중 유독 눈에 띄는 한 남성 출연자의 기상천외한 이름이 밝혀져, 나머지 출연진은 물론 MC들도 당황하며 이름을 재차 물어 웃음을 자아냈다.\n하지만 후에 밝혀진 이름 탄생 배경에는 마냥 웃을 수만은 없는 인생 스토리가 담겨있어 눈길을 끌기도 했다.\n한편, 지난 10월 10일부터 방송된 \'한명회\'는 9회로 막을 내린다.\n오는 12일부터 \'한명회\' 방송 시간에는 편성 시간이 변경된 \'패키지로 세계일주-뭉쳐야 뜬다\'가 전파를 탄다.\n\'한명회\' 마지막 회는 5일 오후 9시 30분에 방송된다.\n[사진 = JTBC 제공]- ⓒ마이데일리(www.mydaily.co.kr).\n무단전재&재배포 금지 -'

In [574]:
sTest2 = sTest.split('\n')
sTest2

["김국진·노홍철·한혜진 '한명회', 오늘(5일) 9회만에 종영.",
 "영상 바로보기 [마이데일리 = 이승길 기자] JTBC '내 이름을 불러줘-한명(名)회'(이하 '한명회')가 막을 내린다.",
 "최근 진행된 '한명회' 9회 녹화는 별난 이름 특집으로 꾸며졌다.",
 '이날은 특집 명에 맞게 녹화 전까지 철저히 출연자들이 서로 이름을 공유하지 못하도록 조치를 취했다.',
 '출연자들은 서로의 이름을 추측하며 자신의 이름보다 특이 한 지 궁금해 했다.',
 '이날 8명의 출연자는 모두 "출석 부르는 시간, 자기소개 시간을 가장 싫어한다"며 다른 이름이지만 같은 심정으로 공감 대화를 나눴다.',
 '그중 유독 눈에 띄는 한 남성 출연자의 기상천외한 이름이 밝혀져, 나머지 출연진은 물론 MC들도 당황하며 이름을 재차 물어 웃음을 자아냈다.',
 '하지만 후에 밝혀진 이름 탄생 배경에는 마냥 웃을 수만은 없는 인생 스토리가 담겨있어 눈길을 끌기도 했다.',
 "한편, 지난 10월 10일부터 방송된 '한명회'는 9회로 막을 내린다.",
 "오는 12일부터 '한명회' 방송 시간에는 편성 시간이 변경된 '패키지로 세계일주-뭉쳐야 뜬다'가 전파를 탄다.",
 "'한명회' 마지막 회는 5일 오후 9시 30분에 방송된다.",
 '[사진 = JTBC 제공]- ⓒ마이데일리(www.mydaily.co.kr).',
 '무단전재&재배포 금지 -']

In [580]:
TextRank(re.sub('[\W]','',sTest)).keywords()

['별난', '기도', '12', '변경', '서로']

* 형태소 분석 : morp
* 어휘의미 분석 : wsd (동음이의어)
* 어휘의미 분석 : wsd_poly (다의어)
* 개체명 인식 : ner
* 의존구문 분석 : dependency
* 의미역 인식 : srl

In [644]:
workrank1 = RunWordKRwordRank(data['title']+'. '+data['mainText'], 3)
workrank1[0]

done


{'9회': 0.8126837885386783,
 '방송': 1.3006717524666578,
 '시간': 1.3519985939266275,
 '이름': 1.9076963967795106,
 '출연자': 0.7325719116229548,
 '한명회': 2.370508665957211}

In [642]:
keyword(etri_outcome,data)

[('한명회', 0.4845437118523489),
 ('이름을', 0.21535276082326621),
 ('막을 내린다', 0.16151457061744967),
 ('5일', 0.16151457061744967),
 ('이름', 0.1076763804116331),
 ('했다', 0.1076763804116331),
 ('막을', 0.1076763804116331)]

[('한명회', 0.71237772470496141),
 ('5일', 0.43204307153704319),
 ('막을 내린다', 0.43204307153704319),
 ('이름을', 0.40594303011854527),
 ('jtbc', 0.28802871435802879)]

In [ ]:
def Extract_Keywords(text, stopwords):
    etri = Run_ETRI_Analysis(stopwords, text)
    soy1 = RunLRNounExtractor(text)
    soy2 = RunWordExtractor(text)
    et0 = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNG','SL','SH','NH'], etri[0]))
    et0 = list(map(lambda x: x[0], et0))
    et1 = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNG','SL','SH','NH'], etri[1]))
    et1 = list(map(lambda x: x[0], et1))
    et2 = list(map(lambda x: x[0], etri[2]))
    et3 = list(map(lambda x: x[0], etri[3]))
    et4 = list(filter(lambda x: x[1] in ['ARG0','ARG1','ARG2','ARG3','ARG4'], etri[4]))
    et4 = list(map(lambda x: x[0], et4))
    et5 = list(filter(lambda x: x[1] in ['NP','NP_SBJ','NP_OBJ','NP_AJT','VNP'], etri[5]))
    et5 = list(map(lambda x: x[0], et5))
    mecabout = list(filter(lambda x: x[1] in ['NNG','NNB','NNP'], mecab.pos(text)))
    ctout = list(filter(lambda x: x[1] == 'Noun' , ct.pos(text)))
    otout = list(filter(lambda x: x[1] == 'Noun' , ot.pos(text)))
    mcout = list(map(lambda x: x[0], mecabout))
    ctout = list(map(lambda x: x[0], ctout))
    otout = list(map(lambda x: x[0], otout))
    out = list(soy1.keys())+list(soy2.keys())+ctout+otout+mcout+et3+et4+et0+et1+et2+et5
    vect = TfidfVectorizer().fit(out)
    y = [list(soy1.keys()), list(soy2.keys()),et3, et4, ctout, otout, mcout,et0, et1, et2, et5]
    y = list(filter(lambda x: len(x) !=0, y))
    outdict = dict()
    for i in y:
        count = vect.transform(i).toarray().sum(axis = 0)
        idx = np.argsort(-count)
        count = count[idx]
        feature_name = np.array(vect.get_feature_names())[idx]
        out = dict(zip(feature_name, count))
        for ii in out:
            if not ii in outdict:
                outdict[ii] = out[ii]
            else:
                outdict[ii] +=out[ii]
    x = sorted(outdict.items(), key = itemgetter(1), reverse=True)[:5]
    output = list(map(lambda x: x[0], x))
    return output


In [ ]:
def Extract_Keyword_from_news_bigdata(path):
    from glob import glob
    fileList = glob(path+'*.csv')
    listIs =list()
    for i in fileList:
        df = pd.read_csv(i,engine='python')
        f = list(filter(lambda x: x[1] in ['중요키워드','키워드','keword','토픽키워드','토픽 키워드'], enumerate(df.columns.values)))
        if not len(f) == 0:
            listIs += list(df[df.columns[f[0][0]]].values)
    outlist = []
    for ix in list(filter(lambda x: type(x)!=float, listIs)):
        outlist += ix.split(',')
    outlist1 = list(map(lambda x: ''.join(x.split('_')), outlist))
    outlist2 = list(map(lambda x: ' '.join(x.split('_')), outlist))
    return outlist1+outlist2

In [171]:
class SentenceTokenizer(object):
    def __init__(self):
        self.twitter = ct
        #self.twitter = ot
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
        ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]    
    def text2sentences(self, text):
        sentences = re.sub('\. ','.\n', text).split('\n')
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.phrases(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
        return nouns
    
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = sknorm(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=5):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords

In [ ]:
for data in useCollection.find({'site':'daum'}):
    if len(data['mainText'].strip()) !=0:
        if data['keywords'] !='NaN':
            ct.add_dictionary(list(map(lambda x: x.split(' · ')[-1], data['keywords'])), 'Noun')
        y = Extract_Keywords2(data['mainText'],stopwords)
        z = TextRank(data['mainText'])
        p = RunLRNounExtractor(data['mainText'])
        q = RunWordExtractor(data['mainText'])
        #print (y)
        print (data['keywords'])
        print (z.keywords())
        #print (p.keys())
        #print (q.keys())
        print ()
        break

# sentiment analysis

In [ ]:
import os
import sys

In [ ]:
def read_data(filename):
    with codecs.open(filename, encoding = 'utf-8', mode='r') as f:
        data